In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

In [24]:
# 크롬 프로필 불러와서 로그인 상태 유지
chrome_options = Options() #옵션 저장
chrome_options.add_argument(r"--user-data-dir=C:\Users\SEUNGJUN\AppData\Local\Google\Chrome_Selenium")  # 내 PC의 chrome 경로에 맞게 경로 조정
chrome_options.add_argument("--profile-directory=Profile 1")  # 보통은 Default, 다른 프로필이면 바꿔야 함

driver = webdriver.Chrome(options=chrome_options)

In [25]:
from datetime import datetime

# 오늘 날짜를 'YYYY-MM-DD(weekday)' 형식으로 가져오기, 예약 날짜 지정
today = datetime.today().strftime('%Y-%m-%d')
weekday_kor = ['월', '화', '수', '목', '금', '토', '일']
weekday = weekday_kor[datetime.today().weekday()]
print(f"{today}({weekday})")

2025-07-25(금)


In [26]:
# 예약 날짜 지정
hope_date = input("YYYY-MM-DD형식으로 입력하세요.")
room_id = "91"
url = f"https://lib.hanyang.ac.kr/facility/room/all-rooms/{room_id}/{hope_date}?hopeDate={hope_date}"
driver.get(url)
time.sleep(2)  # 페이지 로딩 대기

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

lib_id = "########"
lib_pw = "########"

# 아이디 입력
id_input = driver.find_element(By.CSS_SELECTOR, 'input[formcontrolname="userId"]')
id_input.clear()
id_input.send_keys(lib_id)

# 비밀번호 입력
pw_input = driver.find_element(By.CSS_SELECTOR, 'input[formcontrolname="password"]')
pw_input.clear()
pw_input.send_keys(lib_pw)

# 로그인 버튼 활성화 대기 후 클릭
WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, '//button[.//span[text()="로그인"]]'))
)
login_btn = driver.find_element(By.XPATH, '//button[.//span[text()="로그인"]]')
login_btn.click()

In [28]:
from selenium.webdriver.support.ui import Select

# 선택지 딕셔너리
choices = {1: "13:00", 2: "15:00"}

# 사용자에게 번호를 입력받음
num = int(input("예약 시작 시간을 선택하세요 (1: 13:00, 2: 15:00): "))

# 선택한 시간 가져오기
selected_time = choices[num]

# 예약시작시간 셀렉트 박스 요소 찾기
select_begin_elem = driver.find_element(By.CSS_SELECTOR, 'select[formcontrolname="beginTime"]')
select_begin_obj = Select(select_begin_elem)

# 드롭다운에서 시간 선택
try:
    select_begin_obj.select_by_value(selected_time)
except Exception as e:
    print("해당 옵션이 없습니다. 다른 시간대를 선택하세요.")

In [29]:
# 예약종료시간 셀렉트 박스 요소 찾기
select_end_elem = driver.find_element(By.CSS_SELECTOR, 'select[formcontrolname="endTime"]')
select_obj = Select(select_end_elem)

# 드롭다운에서 시간 선택
endtime_map = {"13:00": "15:00", "15:00": "17:00"}
selected_endtime = endtime_map.get(selected_time)
if selected_endtime:
    try:
        select_obj.select_by_value(selected_endtime)
    except Exception as e:
        print("해당 옵션이 없습니다. 다른 시간대를 선택하세요.")
else:
    print("매칭되는 종료 시간이 없습니다.")

In [30]:
# 이용목적 셀렉트 박스 요소 찾기
select_use_elem = driver.find_element(By.CSS_SELECTOR, 'select[formcontrolname="useSection"]')
select_obj = Select(select_use_elem)
select_obj.select_by_value("2") #회의

In [ ]:
# 전체 명단: 이름을 key로, 학번을 value로 저장
all_members = {
    "김##": "2016042606",
    "김##": "2017046353",
    "황##": "2018026299",
    "김##": "2016044293",
    "박##": "2019033454",
    "송##": "2017029043",
    "이##": "2020076571",
    "이##": "2018025750",
    "이##": "2021048813"
}

selected_names = input(
    "동반 이용자 4명의 이름을 ,로 구분해서 입력: \n"
    "목록: 김##, 김##, 황##, 김##, 박##, 송##, 이##, 이##, 이##").split(",")
selected_names = [name.strip() for name in selected_names][:4]  # 앞뒤 공백 제거, 최대 4명

# 동반자 추가 아이콘 클릭 (처음 한 번만)
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'mat-icon[matSuffix]'))
).click()

for name in selected_names:
    if name not in all_members:
        print(f"{name} 정보 없음, 건너뜀.")
        continue
    memberNo = all_members[name]
    
    # 이름 입력
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'input[formcontrolname="name"]'))
    ).clear()
    driver.find_element(By.CSS_SELECTOR, 'input[formcontrolname="name"]').send_keys(name)
    time.sleep(0.5)
    
    # 학번 입력
    driver.find_element(By.CSS_SELECTOR, 'input[formcontrolname="memberNo"]').clear()
    driver.find_element(By.CSS_SELECTOR, 'input[formcontrolname="memberNo"]').send_keys(memberNo)
    
    # "추가" 버튼 클릭
    driver.find_element(By.XPATH, '//button[span[text()="추가"]]').click()
    time.sleep(2.5)  # 너무 빨리 다음 반복 돌지 않게

print("동반 이용자 4명 추가 완료!")


동반 이용자 4명 추가 완료!


In [32]:
#한번 확인하고, 등록 클릭!

driver.find_element(By.XPATH, '//button[span[text()="등록"]]').click()

In [33]:
#동반이용자 개인정보수집동의 클릭 단 클릭이 가능해질때까지 대기.

try:
    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//label[contains(text(),"위 사항을 확인하고 동의합니다")]'))).click()
except:
    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//label[contains(text(),"위 사항을 확인하고 동의합니다")]'))).click()

In [34]:
# 마지막 신청버튼 클릭

driver.find_element(By.XPATH, '//button[.//span[text()="신청"]]').click()

In [35]:
# 예약화면 캡쳐

driver.get('https://lib.hanyang.ac.kr/mylibrary/room/reservations')
time.sleep(2)

# (A) 모든 예약의 펼침 버튼 클릭
expand_buttons = driver.find_elements(By.CSS_SELECTOR, ".mat-expansion-indicator")
try: 
    for btn in expand_buttons:
        btn.click()
        time.sleep(1)
except:
    None

save_path = fr"C:\Users\SEUNGJUN\Pictures\도서관예약내역\도서관예약_{hope_date}.png"
driver.save_screenshot(save_path)
print("캡쳐화면 저장 완료!")


캡쳐화면 저장 완료!


In [36]:
driver.quit()

In [37]:
# 개인정보로 저장 google drive와 연동
# google_client_id = "1046852879307-s1qtbaaq011ehmc4skc993io66eejjs9.apps.googleusercontent.com"
# google_client_pw = "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"

google_drive_folder_id = "1pXzpCogQaDfyuf_i6aytAnPEjXSsWQPU"
google_client_secret_file = r'C:\Users\SEUNGJUN\project_file\민감정보\client_secret_1046852879307-s1qtbaaq011ehmc4skc993io66eejjs9.apps.googleusercontent.com.json'
scopes = ['https://www.googleapis.com/auth/drive.file'] # 구글 드라이브 연동 설정

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle

def get_gdrive_service():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                google_client_secret_file, scopes)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('drive', 'v3', credentials=creds)

def upload_file_to_gdrive(service, folder_id, file_path):
    filename = os.path.basename(file_path)
    file_metadata = {
        'name': filename,
        'parents': [folder_id]
    }
    media = MediaFileUpload(file_path, resumable=True)
    uploaded = service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id,webViewLink'
    ).execute()
    # 누구나 볼 수 있도록 공개 설정
    service.permissions().create(
        fileId=uploaded['id'],
        body={'type': 'anyone', 'role': 'reader'}
    ).execute()
    print('업로드 완료! 파일 링크:', uploaded['webViewLink'])
    return uploaded['webViewLink']

# 실제 사용 예시
service = get_gdrive_service()
link = upload_file_to_gdrive(service, google_drive_folder_id, save_path)
print(link)   # 이걸 카카오톡 메시지에 첨부
     

업로드 완료! 파일 링크: https://drive.google.com/file/d/139s3HpnT4FRKjcBUFZm74wvLSMVV1GPi/view?usp=drivesdk
https://drive.google.com/file/d/139s3HpnT4FRKjcBUFZm74wvLSMVV1GPi/view?usp=drivesdk


In [38]:
REST_API_KEY = '918871529f1430cbf8a1ff62a74ba869'
REDIRECT_URI = 'https://example.com/oauth'  # 예: https://localhost:3000
CLIENT_SECRET = 'mhZ1yXNXzJFZP3REWL4lTndplvvGYYjc'

auth_url = (
    f"https://kauth.kakao.com/oauth/authorize?"
    f"client_id={REST_API_KEY}&redirect_uri={REDIRECT_URI}&response_type=code&scope=talk_message"
)
print(auth_url)

https://kauth.kakao.com/oauth/authorize?client_id=918871529f1430cbf8a1ff62a74ba869&redirect_uri=https://example.com/oauth&response_type=code&scope=talk_message


In [39]:
import requests
import json

with open(r"C:\Users\SEUNGJUN\project_file\민감정보\kakao_code.json","r") as fp:
    tokens = json.load(fp)
# 결과: {'access_token': '...', 'refresh_token': '...', ...}

REFRESH_TOKEN = tokens["refresh_token"]

# 2. refresh_token으로 access_token을 재발급
url = "https://kauth.kakao.com/oauth/token"
data = {
    "grant_type": "refresh_token",
    "client_id": REST_API_KEY,
    "refresh_token": REFRESH_TOKEN,
    "client_secret": CLIENT_SECRET
}
response = requests.post(url, data=data)
new_tokens = response.json()
print(new_tokens)

# 3. 새로 발급된 access_token이 있으면 기존 토큰 정보 갱신 (refresh_token도 갱신될 수 있음)
tokens["access_token"] = new_tokens["access_token"]
if "refresh_token" in new_tokens:
    tokens["refresh_token"] = new_tokens["refresh_token"]

# 4. 다시 저장
with open(r"C:\Users\SEUNGJUN\project_file\project_file\kakao_code.json", "w") as fp:
    json.dump(tokens, fp)

{'access_token': 'tKTNAR9sWdFt8EDshl6oRZ7d4WPi2J6aAAAAAQoNGVMAAAGYPcfBsPsTCyemC-i_', 'token_type': 'bearer', 'expires_in': 21599}


In [40]:
def kakao_token_first_issue(CODE, REST_API_KEY, REDIRECT_URI, CLIENT_SECRET, save_path):
    url = "https://kauth.kakao.com/oauth/token"
    data = {
        "grant_type": "authorization_code",
        "client_id": REST_API_KEY,
        "redirect_uri": REDIRECT_URI,
        "code": CODE,
        "client_secret": CLIENT_SECRET
    }
    response = requests.post(url, data=data)
    tokens = response.json()
    print(tokens)
    with open(save_path, "w") as fp:
        json.dump(tokens, fp)
# 결과: {'access_token': '...', 'refresh_token': '...', ...}

In [41]:
def send_kakao_me(access_token, hope_date, link):
    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    # 일반 텍스트에 URL 포함 - 카카오톡이 자동으로 미리보기 생성
    message_text = f"""한양대 도서관 예약 완료! 📚

📅 예약일: {hope_date}
✅ 시설예약이 완료되었습니다.

📋 예약내역 확인하기:
{link}"""
    
    data = {
        "template_object": json.dumps({
            "object_type": "text",
            "text": message_text,
            "link": {
                "web_url": link,
                "mobile_web_url": link
            }
        })
    }
    
    resp = requests.post(url, headers=headers, data=data)
    print(resp.json())
    
    if resp.json().get('result_code') == 0:
        print('메시지를 성공적으로 보냈습니다.')
    else:
        print(f'메시지를 성공적으로 보내지 못했습니다. 오류메시지 : {resp.json()}')
    
    return resp.json()

In [42]:
send_kakao_me(tokens["access_token"],hope_date,link)

{'result_code': 0}
메시지를 성공적으로 보냈습니다.


{'result_code': 0}